In [1]:
import numpy as np
import cv2

In [2]:
cap = cv2.VideoCapture(0)

ret,frame = cap.read()

face_cascade = cv2.CascadeClassifier('DATA/haarcascades/haarcascade_frontalface_default.xml')
face_rects = face_cascade.detectMultiScale(frame)

# Convert this list of a single array to a tuple of (x,y,w,h)
(face_x,face_y,w,h) = tuple(face_rects[0])
track_window = (face_x,face_y,w,h)

# set up the ROI for tracking
roi = frame[face_y:face_y+h, face_x:face_x+w]

hsv_roi = cv2.cvtColor(roi,cv2.COLOR_BGR2HSV)

# FIND HISTOGRAM TO BACKPROJECT THE TARGET ON EACH FRAME IN ORDER TO CALCULATE THE MEAN SHIFT
roi_hist = cv2.calcHist([hsv_roi],[0],None,[180],[0,180])

# Normalize the histogram array values given a min of 0 and max of 255
cv2.normalize(roi_hist,roi_hist,0,255,cv2.NORM_MINMAX)

# termination criteria
term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT,10,1) # going for 1 episolen or 10 iteratation

while True:
    
    ret, frame = cap.read()
    
    if ret == True:
        
         # Grab the Frame in HSV
        hsv = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
        
        # CALUCULATE THE BACK PROJECTION BASED ON THE ROI HIST THAT WE CREATED EARLIER
        dst = cv2.calcBackProject([hsv],[0],roi_hist,[0,180],1)
        
        #  Apply meanshift to get the new coordinates of the rectangle
        ret, track_window = cv2.meanShift(dst,track_window,term_crit)
        
        # draw a new tracking window based on track_window updated
        x,y,w,h = track_window
        img2 = cv2.rectangle(frame,(x,y),(x+w,y+h),(0,0,255),5)
        
        cv2.imshow('img',img2)
        
        k = cv2.waitKey(1) & 0xFF
        if k == 27:
            break
            
    else:
        break
        
        

cv2.destroyAllWindows()
cap.release()
    
    
    


AttributeError: 'cv2.VideoCapture' object has no attribute 'relase'